In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import Image
from IPython.display import Markdown
def get_csv_resources(package_id):
    """
    Retrieve the CSV resources associated with a package.

    Parameters:
    - package_id (str): The ID of the package containing the resources.

    Returns:
    - list: A list of CSV resource dictionaries.
    """
    base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
    url = f"{base_url}/api/3/action/package_show"
    params = {"id": package_id}
    package = requests.get(url, params=params).json()
    resources = package.get("result", {}).get("resources", [])
    csv_resources = [res for res in resources if res["format"].lower() == "csv"]
    return csv_resources

def retrieve_csv_data(resource_url):
    """
    Retrieve CSV data from a CSV resource URL.

    Parameters:
    - resource_url (str): The URL of the CSV resource.

    Returns:
    - pd.DataFrame or None: A DataFrame containing the CSV data, or None if retrieval failed.
    """
    response = requests.get(resource_url)
    if response.status_code == 200:
        csv_data = response.text
        df = pd.read_csv(StringIO(csv_data))
        return df
    else:
        print(f"Failed to retrieve CSV data from {resource_url}")
        return None

def main():
    package_id = "library-branch-programs-and-events-feed"
    csv_resources = get_csv_resources(package_id)

    for idx, resource in enumerate(csv_resources):
        csv_url = resource["url"]
        df = retrieve_csv_data(csv_url)

        if df is not None:
            # Rename columns to lowercase
            df.columns = map(str.lower, df.columns)
            
            # Filter out rows where library is "All Branches"
            df = df[df['library'] != 'All Branches']
            
            # Extract library names and sort them alphabetically
            libraries = df['library'].unique().tolist()
            libraries.sort()

            # Define a function to handle dropdown selection
            def on_change(library):
                # Filter DataFrame based on selected library
                filtered_df = df[df['library'] == library]
                
                # Get all possible age group values
                all_age_groups = sorted(df['agegroup1'].unique())
                
                # Aggregate event counts by age group
                library_counts = filtered_df.groupby('agegroup1').size()
                
                # Create a DataFrame with all possible age groups and fill missing values with 0
                library_counts = library_counts.reindex(all_age_groups, fill_value=0)

                # Set Seaborn style
                sns.set_style("whitegrid")

                # Plot bar chart
                plt.figure(figsize=(7, 4))
                plt.bar(library_counts.index, library_counts.values)
                
                # Set font size and style for title
                plt.title(f'Event Counts by Age Group for {library}', fontsize=20, fontweight='bold', pad=20)

                # Set font size for x and y axis labels
                plt.xlabel('Age Group', fontsize=16, labelpad=25)
                plt.ylabel('Event Count', fontsize=16, labelpad=25)

                # Set font size for x-axis ticks
                plt.xticks(rotation=45, ha='right', fontsize=12)

                # Set font size for y-axis ticks
                plt.yticks(fontsize=12)

                # Customize grid using Matplotlib
                plt.grid(True, linestyle=':', color='lightgrey', zorder=0)

                # Display the image
                display(Image("Toronto_Public_Library_Logo.png", width=500, height=200))

                # Display text area with custom font size, style, and boldness
                text_area_text = "<h2>TPL is responsible for the effective <br> management of the quality of Library services, <br>numerous events for every age group and many more..</h2>"
                display(Markdown(text_area_text))


                # Display URL link area
                url_link_text = "[Link to events of Library Website](https://events.tplcs.ca/)"
                display(Markdown(url_link_text))

                # Show the plot
                plt.show()

            # Register event handler for dropdown widget with custom description
            interactive_plot = widgets.interactive(on_change, library=widgets.Dropdown(options=libraries, description="Select Library"))
            display(interactive_plot)
            
            # Break out of the loop after processing the first resource
            break
        else:
            print("No data retrieved from this CSV resource.")

if __name__ == "__main__":
    main()